In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ChatBot_with_chain_Prompt_template"

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

# Build the Prompt

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
# Build the chain
chain = prompt | model

Note that this slightly changes the input type - rather than pass in a list of messages, we are now passing in a dictionary with a messages key where that contains a list of messages.

In [5]:
from langchain_core.messages import HumanMessage

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"})

In [6]:
response

AIMessage(content='¡Hola, Bob! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-249a6a36-4c84-4cc1-944f-1f2f517c27f7-0')

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. 


# Define message store


In [7]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# Function to get session history
This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


In [8]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [9]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [10]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

# Define Config

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. 

In [11]:
config_1 = {"configurable": {"session_id": session_id_1}}

In [12]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config_1,
)

In [13]:
response

AIMessage(content='¡Hola, Todd! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f3db212e1c', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e671cb9-0c01-4150-b4e6-2ecaf808b05f-0')

In [14]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_1,
)

In [15]:
response

AIMessage(content='आपका नाम टॉड है।', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 56, 'total_tokens': 64}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-af934fd9-b45c-491b-82a1-5f87e88cda68-0')

In [16]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What was my last question?")], "language": "English"},
    config=config_1,
)

In [17]:
response

AIMessage(content='Your last question was, "What\'s my name?"', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 78, 'total_tokens': 88}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-af9e9980-138d-449c-8bd8-95b140f090c7-0')

In [18]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "What was my last question?"\n2. "What’s my name?"', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 104, 'total_tokens': 127}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-bacec727-7799-4c8c-a9e5-7d118e2dd62a-0')

In [19]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "What was my last question?"\n2. "What’s my name?"', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 143, 'total_tokens': 166}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_db4a9208a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-69038af2-0fb3-49c6-9e74-a2fb233c6e2d-0')

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 70 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content="I can only recall the context of our current conversation, which includes your last few questions. I don’t have the capability to access or remember previous interactions beyond this session. If you have specific questions or topics you'd like to revisit, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 182, 'total_tokens': 233}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-33a8e088-d4de-45de-9c2f-c7393246f30e-0')

In [21]:
# Generating a dynamic session ID
session_id_2 = generate_session_id()
config_2 = {"configurable": {"session_id": session_id_2}}

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name? Tell me a joke")], "language": "English"},
    config=config_1,
)
response

AIMessage(content="Your name is Todd. Here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 266, 'total_tokens': 289}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-01d4514e-59ed-4b09-960d-bdccef496ccb-0')

In [31]:
response.content

"Your name is Todd. Here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

# Streaming final outputs
The .stream method will by default stream each key in a sequence.

Note that here only the "answer" key is streamed token-by-token, as the other components-- such as retrieval-- do not support token-level streaming.

In [40]:
stream = with_message_history.stream(
    {"messages": [HumanMessage(content="Whats my name? Tell me a joke")], "language": "Engling"},
    config=config_1,
)

In [41]:
for chunk in stream:
    print(f"{chunk}", end="")

content='' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content='Your' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' name' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' is' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' Todd' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content='.' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' Here' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content='’s' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' a' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' joke' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' for' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' you' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=':\n\n' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content='Why' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' did' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' the' id='run-8a0873c9-cb69-4f27-8bdc-a3de47383b40'content=' scare' id='run-8a0873c9-cb69-4f27-8bdc

In [42]:
stream = with_message_history.stream(
    {"messages": [HumanMessage(content="Whats my name? Tell me a joke")], "language": "Engling"},
    config=config_1,
)

In [43]:
for chunk in stream:
    print(chunk.content, end="|")

|Your| name| is| Todd|.| Here|’s| another| joke| for| you|:

|Why| did| the| bicycle| fall| over|?

|Because| it| was| two|-t|ired|!||

In [44]:
stream = with_message_history.stream(
    {"messages": [HumanMessage(content="Whats my name? Tell me a joke")], "language": "Engling"},
    config=config_1,
)

In [45]:
for chunk in stream:
    print(chunk.content, end="")

Your name is Todd. Here’s a joke for you:

Why did the math book look sad?

Because it had too many problems!